# `g++` compiler options

## In a nutshell

Writing reliable and portable numerical programs in C++ requires care.

It is a good practice to compile and test your program with (at least)
two different compilers.

The following minimum `g++` (GNU `C++` compiler) options are recommended when compiling numerical programs:

```shell
g++ -Werror -Wall -Wextra -Wshadow -g -O2 -o <filename>
```

The following list gives a brief explanation of these compiler options:

- `-Werror`: Consider warnings to be errors, so that compilation stops when a warning occurs. This prevents warnings from scrolling off the top of the screen and being
lost. You won’t be able to compile the program until it is completely
warning-free. You may be tempted to remove this. DON’T!
- `-Wall`: This turns on a set of warnings for common programming problems. You
need `-Wall`, but it is not enough on its own (for g++).
- `-Wextra`: This turns on some extra warnings in g++ not included in `-Wall`, such
as missing return values and comparisons between signed and unsigned
integers.
- `-Wshadow`: This warns whenever a local variable shadows another local variable.
If two variables have the same name then it is a potential source of
confusion.
- `-O2`: Turn on optimization. The warnings for uninitialized variables in `-Wall` rely on the optimizer to analyze the code. If there is no
optimization then the warnings aren’t generated. When first debugging,
use `-Og` (optimizing the debugging experience) until ready for production runs.
- `-g`: It always makes sense to put debugging symbols in the executable so
that you can debug it using debuggers such as `gdb`. The only effect of debugging symbols
is to increase the size of the file, and you can use the [`strip`](https://man7.org/linux/man-pages/man1/strip.1.html)
command to remove them later for production runs of your source if necessary. 
You can also use `ggdb` to produce debugging information for use by GDB. This means to use the most expressive format available (DWARF, stabs, or the native format if neither of those are supported), including GDB extensions if at all possible.

- `-o <filename>`: writes the output to filename (executable).

For a complete list, see the [GCC Command Options](https://gcc.gnu.org/onlinedocs/gcc/Invoking-GCC.html) documentation.

Turning on [optimization flags](https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html) makes the compiler attempt to improve the performance and/or code size at the expense of compilation time and possibly the ability to debug the program:

- `-O0`: Reduce compilation time and make debugging produce the expected results. This is the default.
- `-O1`, `-O2`, `-O3`: increases the optimization level, increasing compilation time and performance 
- `-Ofast`: Disregard strict standards compliance. `-Ofast` inlcludes all `-O3` optimizations and turns on `-ffast-math`.
- `-Og`: Optimize debugging experience. `-Og` should be the optimization level of choice for the standard edit-compile-debug cycle, offering a reasonable level of optimization while maintaining fast compilation and a good debugging experience. It is a better choice than `-O0` for producing debuggable code because some compiler passes that collect debug information are disabled at `-O0`. 
- `-ffast-math`: This option can result in incorrect output for programs that depend on an exact implementation of IEEE or ISO rules/specifications for math functions. It may, however, yield faster code for programs that do not require the guarantees of these specifications. Be careful! 

Now, let's test these compiler flags in practice, especially those for warnings. Let's write a simple `C++` code that should result in warnings:

In [13]:
%%file testing_compiler_flags.cpp
double global = 3.14;
int main(){
    int a = 3;
    unsigned b = -4;
    double global = 4;
    if (a == b){
        global = 2;
    }
    int c = 20;
    int d;
    // return 0;
}

Overwriting testing_compiler_flags.cpp


By construction, this simple code has several issues. Go to your terminal and compile the code:
```shell
export CXXFLAGS="-Werror -Wall -Wextra -Wshadow -g -O2"
g++ $CXXFLAGS -o testing_compiler_flags testing_compiler_flags.cpp
```
What warnings do you expect? Experiment with different flags defined in the variable `CXXFLAGS`. 

## More details

The following is an extended list of `g++` warning options, which catch
less common problems and ensure greater portability:

```shell
g++ -Werror -Wall -W -Wshadow -fno-common -g -O3
-ansi -pedantic -Wconversion -Wpointer-arith -Wcast-qual
-Wcast-align -Wwrite-strings -fshort-enums
```

The following list gives a brief explanation of these compiler options:

- `-Wpedantic`: `-std=c++03` [other ISO standards include `-std=c++11` and `-std=c++14`]
Use ISO C++, and reject any non-ANSI extensions. These flags help in
writing portable programs that will compile on other systems.
- `-Wconversion`: The main use of this option is to warn about implicit conversions that may alter a value, including conversins between real and integer, like `abs(x)` for real `x` and conversions between signed
and unsigned integers such as `unsigned int x = -1`. Explicit casts will not result in warnings.
- `--Wcast-qual -Wcast-align`
These options warn if you try to do pointer arithmetic for types
which don’t have a size, such as `void`, if you remove a `const cast`
from a pointer, or if you cast a pointer to a type which has a
different size, causing an invalid alignment.
- `-Wwrite-strings`
This option gives string constants a const qualifier so that it will
be a compile-time error to attempt to overwrite them.
- `-fshort-enums`
This option makes the type of `enum` as short as possible. Normally,
this makes an `enum` different from an `int`. Consequently any attempts
to assign a pointer-to-int to a pointer-to-enum will generate a
cast-alignment warning.
- `-Wundef`: Warn if an undefined identifier is evaluated in an `#if` preprocessor directive. Such identifiers are replaced with zero.
  
See also the [C++ Warnings Test Suite](https://github.com/fleschutz/CWTS).

## Useful environment variables

* `LIBRARY_PATH`: The value of `LIBRARY_PATH` is a colon-separated list of directories, much like PATH. When configured as a native compiler, GCC tries the directories thus specified when searching for special linker files, if it cannot find them using `GCC_EXEC_PREFIX`. Linking using GCC also uses these directories when searching for ordinary libraries for the `-l` option (but directories specified with `-L` come first).
* `CPATH`, `C_INCLUDE_PATH`, `CPLUS_INCLUDE_PATH`, `OBJC_INCLUDE_PATH`: Each variable’s value is a list of directories separated by a special character, much like `PATH`, in which to look for header files. The special character, `PATH_SEPARATOR`, is target-dependent and determined at GCC build time. For Microsoft Windows-based targets it is a semicolon, and for almost all other targets it is a colon. `CPATH` specifies a list of directories to be searched as if specified with `-I`, but after any paths given with `-I` options on the command line. This environment variable is used regardless of which language is being preprocessed. The remaining environment variables apply only when preprocessing the particular language indicated. Each specifies a list of directories to be searched as if specified with `-isystem`, but after any paths given with `-isystem` options on the command line. In all these variables, an empty element instructs the compiler to search its current working directory. Empty elements can appear at the beginning or end of a path. For instance, if the value of `CPATH` is `:/special/include`, that has the same effect as `-I. -I/special/include`.


## Acknowledgments

This document is partly based on the [GCC Documentation](https://gcc.gnu.org/onlinedocs/gcc/) documentation and Dick Furnstahl's [recommended C++ options](https://www.asc.ohio-state.edu/physics/ntg/6810/handouts/recommended_C++_options.pdf).